Montero Barraza Álvaro David

Tecnologías de lenguaje natural 5BV1

Ingeniería en IA

Este programa identifica frases, palabras y documentos similares



## **Modulos**

In [1]:
import spacy
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
from copy import deepcopy
import re
import random
from keybert import KeyBERT
from collections import Counter

/home/alvaromontero/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alvaromontero/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-06-04 22:55:10.993755: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 22:55:11.213505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one h

## **Funciones**

In [2]:
def randomize(lista):
    randomized = lista[:]  
    random.shuffle(randomized)
    return randomized

# Elimina etiquetas HTML y comentarios dentro del propio texto como indicaciones extra entre parentesis o comentarios entre --
def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-z\s]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc
    
# Removemos stop words unicamente en ingles
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Lematizacion mediante spacy
nlp = spacy.load("en_core_web_trf")
def lemmatize(doc):
    docs = list(nlp.pipe([doc]))
    lemas = ' '.join([token.lemma_ for doc in docs for token in doc])
    return lemas

## **Análisis exploratorio y preprocesamiento**

In [3]:
Book_list=['/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/ancient_egypt.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/christ_roma.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/city_rome.txt',
           '/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/early_european.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/EGYPT.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/greek_cities.txt',
           '/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/greek_imp.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/Punic_war.txt',
           '/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/religion_pagan.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/roman_religion.txt']
#Book_list=[]

Books_raw_file=[]
for i in range(len(Book_list)):
    with open(Book_list[i],'r',encoding='utf-8')as f:
        file=f.read()
        Books_raw_file.append(file)

In [4]:
if spacy.prefer_gpu():
    print("El modelo está configurado para usar GPU")
else:
    print("El modelo está configurado para usar CPU")

El modelo está configurado para usar CPU


In [5]:

nlp=spacy.load('en_core_web_trf')
spacy.prefer_gpu()
books=[]
books_w_stops=[]
for i,book in enumerate(Books_raw_file):
    book=eliminar_etiquetas(book)
    book=remover_apostrofes(book)
    #book=remover_especiales(book)
    book=dobles_espacios(book)
    
    doc=nlp(book)
    books_w_stops.append(doc.copy())#Libros con stopwords
    
    book=remove_stopwords(book)
    doc_s=nlp(book)
    books.append(doc_s.copy())#Libros sin stopwords
    print(f"Libro {i} {len(doc)} tokens  {len(list(doc.sents))} oraciones")

Libro 0 5813 tokens  174 oraciones
Libro 1 14631 tokens  458 oraciones
Libro 2 1217 tokens  60 oraciones
Libro 3 4447 tokens  261 oraciones
Libro 4 14737 tokens  431 oraciones
Libro 5 10615 tokens  314 oraciones
Libro 6 9525 tokens  304 oraciones
Libro 7 4313 tokens  174 oraciones
Libro 8 8404 tokens  281 oraciones
Libro 9 8629 tokens  294 oraciones


In [6]:
print("Sin stopwords")
for i,book in enumerate(books):
    print(f"Libro {i} {len(book)} tokens  {len(list(book.sents))} oraciones")

Sin stopwords
Libro 0 3626 tokens  182 oraciones
Libro 1 9104 tokens  512 oraciones
Libro 2 681 tokens  62 oraciones
Libro 3 3035 tokens  266 oraciones
Libro 4 8824 tokens  449 oraciones
Libro 5 6618 tokens  330 oraciones
Libro 6 5596 tokens  318 oraciones
Libro 7 2521 tokens  178 oraciones
Libro 8 5044 tokens  288 oraciones
Libro 9 5417 tokens  366 oraciones


## **Desarrollo de la práctica**

Similitud de palabras con synsets

In [15]:
for i,book in enumerate(books):
    print(book[10].pos_)

ADJ
NOUN
NOUN
NOUN
PUNCT
PUNCT
VERB
PROPN
NOUN
PROPN


In [ ]:
def most_common_verb(doc):
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]
    if not verbs:
        return None
    most_common = Counter(verbs).most_common(1)[0][0]
    return most_common

def get_synsets(word, lang='eng'):
    return wn.synsets(word, lang=lang, pos=wn.VERB)

def find_similar_verbs(target_verb, verbs, similarity_func):
    target_synsets = get_synsets(target_verb)
    if not target_synsets:
        return []
    target_synset = target_synsets[0]
    
    similarities = []
    for verb in verbs:
        verb_synsets = get_synsets(verb)
        if not verb_synsets:
            continue
        verb_synset = verb_synsets[0]
        similarity = similarity_func(target_synset, verb_synset)
        if similarity is not None:
            similarities.append((verb, similarity))
    
    # Ordenar por similitud y obtener los 5 más similares
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return [verb for verb, _ in similarities[:5]]

